## Load Dependencies

In [1]:
!pip install -q langchain langchain-openai langchain_community langchainhub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00


In [13]:
# import the necessary modules
import getpass
import os

from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

from google.colab import userdata

# provide the API key
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY4')

In [20]:
# create openai chat model
model = ChatOpenAI(model="gpt-4o")

# create basic string output parser
parser = StrOutputParser()

## Zero-Shot Prompting

In [4]:
# create prompt template
system_template_zero_shot = "Translate the following english {text} into spanish:"

prompt_template_zero_shot = ChatPromptTemplate.from_messages(
    [("system", system_template_zero_shot), ("user", "{text}")]
)

# using LCEL to create chain
chain_zero_shot = prompt_template_zero_shot | model | parser

# invoke the chain
chain_zero_shot.invoke({"text": "Hello how are you today ?"})

'Hola, ¿cómo estás hoy?'

## Few-Shot Prompting

In [6]:
# create prompt template
system_template_few_shot = """
Classify the following feedback into 3 sentiment categories (positive, neutral, and negative):

Here is some examples for your reference:

feedback: The product is amazing and exceeded my expectations
output: positive

feedback: The delivery was late, and the packaging was damaged
output: negative

feedback: The product is okay, but the service could be better
output: neutral

Classify this feedback:

feedback: {user_feedback}
output:
"""

prompt_template_few_shot = ChatPromptTemplate.from_messages(
    [("system", system_template_few_shot), ("user", "{user_feedback}")]
)

# using LCEL to create chain
chain_few_shot = prompt_template_few_shot | model | parser

# invoke the chain
chain_few_shot.invoke({"user_feedback": "this keyboard is very good"})

'positive'

## Chain of Tought

In [7]:
# create prompt template
system_template_cot = """
You need to answer this user question:

user question: {question}
output:

Lets think step by step
"""

prompt_template_cot = ChatPromptTemplate.from_messages(
    [("system", system_template_cot), ("user", "{question}")]
)

# using LCEL to create chain
chain_cot = prompt_template_cot | model | parser

question = """
You have two options for travel.
- Option A costs $200 and takes 4 hours.
- Option B costs $150 but takes 6 hours.
Which is better in terms of cost per hour?
"""

# invoke the chain
chain_cot.invoke({"question": question})

"Let's calculate the cost per hour for each option:\n\n- **Option A**: \n  - Cost: $200\n  - Time: 4 hours\n  - Cost per hour = Total cost / Total time = $200 / 4 hours = $50 per hour\n\n- **Option B**: \n  - Cost: $150\n  - Time: 6 hours\n  - Cost per hour = Total cost / Total time = $150 / 6 hours = $25 per hour\n\nIn terms of cost per hour, Option B is better as it costs $25 per hour compared to Option A which costs $50 per hour."

## ReAct

In [16]:
# create tools, for this purpose we dont use any tools
tools = []

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

# Construct the ReAct agent
agent = create_react_agent(model, tools, prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input" : "how to use PS4 controller in steam?"})

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
To use a PS4 controller in Steam, you need to follow these steps:

1. **Connect Your Controller**: You can connect the PS4 controller to your PC using a USB cable or via Bluetooth. If you're using Bluetooth, ensure that your PC has Bluetooth capability.

2. **Enable Steam Input**: 
   - Open Steam and go to "Steam" in the top-left corner.
   - Select "Settings" from the drop-down menu.
   - In the settings menu, click on "Controller" and then "General Controller Settings."
   - Check the box for "PlayStation Configuration Support." This allows Steam to recognize and use your PS4 controller.

3. **Customize Your Controller (Optional)**:
   - In the same "General Controller Settings" menu, you can customize the button layout and other settings if needed.

4. **Launch Your Game**: 
   - Start the game you want to play, and the PS4 controller should work automatically.
   - Some games might have specific controller settings within their own menus, so

{'input': 'how to use PS4 controller in steam?',
 'output': 'To use a PS4 controller in Steam, connect it via USB or Bluetooth, enable "PlayStation Configuration Support" in Steam\'s controller settings, and your controller should work with Steam games.'}

## ToT

In [21]:
# create prompt template ref: https://github.com/dave1010/tree-of-thought-prompting
system_template_tot = """
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is: {question}
"""

prompt_template_tot = ChatPromptTemplate.from_messages(
    [("system", system_template_tot), ("user", "{question}")]
)

# using LCEL to create chain
chain_tot = prompt_template_tot | model | parser

# invoke the chain
chain_tot.invoke({"question": "why the sky is blue ?"})

"Expert 1: The sky appears blue to us because of the way Earth's atmosphere scatters sunlight.\n\nExpert 2: Sunlight is made up of different colors of light, each with its own wavelength. When sunlight enters Earth's atmosphere, it collides with air molecules and other small particles.\n\nExpert 3: Among the colors of the visible spectrum, blue light has a shorter wavelength and is scattered in all directions by the gases and particles in the atmosphere more than other colors.\n\n---\n\nExpert 1: This scattering process is known as Rayleigh scattering, named after the British scientist Lord Rayleigh who first described it.\n\nExpert 2: Since blue light is scattered in all directions, it is much more likely to reach our eyes, making the sky appear predominantly blue.\n\nExpert 3: During sunrise and sunset, the sky can appear red or orange because the sun is lower in the sky and its light passes through more atmosphere, scattering the shorter wavelengths and leaving the longer wavelength